# Customers Who Bought This Frequently Buy This!
In this notebook we will explore which items were frequently purchased together. Using this information, we can predict which items a customer will buy after we observe what they have already bought!

In [ ]:
import cudf, gc
import cv2, matplotlib.pyplot as plt
from os.path import exists
print('RAPIDS version',cudf.__version__)

# Load Tranactions

In [ ]:
# LOAD TRANSACTIONS DATAFRAME
df = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
print('Transactions shape',df.shape)
display( df.head() )
date = df[['t_dat']]
display(date.head())
# REDUCE MEMORY OF DATAFRAME
df = df[['customer_id','article_id']]
df.customer_id = df.customer_id.str[-16:].str.hex_to_int().astype('int64')
df.article_id = df.article_id.astype('int32')
display(df.head())
_ = gc.collect()

# Find Items Purchased Together
We will use RAPID cuDF to speed up the dataframe search commands below

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
vc = df.article_id.value_counts()
pairs = {}
for j,i in enumerate(vc.index.values[1000:1050]):
    #if j%10==0: print(j,', ',end='')
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    pairs[i.item()] = [vc2.index[0], vc2.index[1], vc2.index[2]]
    

In [ ]:
items = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
BASE = '../input/h-and-m-personalized-fashion-recommendations/images/'

for i,(k,v) in enumerate( pairs.items() ):
    name1 = BASE+'0'+str(k)[:2]+'/0'+str(k)+'.jpg'
    name2 = BASE+'0'+str(v[0])[:2]+'/0'+str(v[0])+'.jpg'
    name3 = BASE+'0'+str(v[1])[:2]+'/0'+str(v[1])+'.jpg'
    name4 = BASE+'0'+str(v[2])[:2]+'/0'+str(v[2])+'.jpg'
    if exists(name1) & exists(name2) & exists(name3) & exists(name4):
        plt.figure(figsize=(20,5))
        img1 = cv2.imread(name1)[:,:,::-1]
        img2 = cv2.imread(name2)[:,:,::-1]
        img3 = cv2.imread(name3)[:,:,::-1]
        img4 = cv2.imread(name4)[:,:,::-1]
        plt.subplot(1,4,1)
        plt.title('When customers buy this',size=18)
        plt.imshow(img1)
        plt.subplot(1,4,2)
        plt.title('They buy this',size=18)
        plt.imshow(img2)
        plt.subplot(1,4,3)
        plt.title('They buy this',size=18)
        plt.imshow(img3)
        plt.subplot(1,4,4)
        plt.title('They buy this',size=18)
        plt.imshow(img4)
        
        plt.show()
    #if i==63: break

In [ ]:
Matrix = [[] for i in range(100)]
_ = gc.collect()
dt = []
for m, i in enumerate(vc.index.values[0:100]):
    dt.append(df.loc[(df.article_id == i.item()),'customer_id'].unique())
for m, i in enumerate(vc.index.values[0:100]):
    for n, j in enumerate(vc.index.values[0:100]):        
        d = cudf.concat([dt[m], dt[n]]).value_counts()
        Matrix[m].append(len(d.loc[d.values == 2]))


In [ ]:

import matplotlib.pyplot as plt
M = [[[Matrix[i][j],0,0] for i in range(10)] for j in range(10)]# change 10 to any number < 100

from PIL import Image
import numpy as np
M = np.array(M)
M = M / 20000.
plt.imshow(M)

In [ ]:
#Get the article_freq
label_list = range(len(vc.values[:50000:1000]))
label_list = np.array(label_list)
cnt_list = list(int(i) for i in vc.values[:50000:1000])
#print(cnt_list)
cnt_list = np.array(cnt_list)
rects1 = plt.bar(x=label_list, height=cnt_list, width=0.3, alpha=1., color='blue')
plt.ylim(0, 60000)     # y轴取值范围
plt.ylabel("")
"""
设置x轴刻度显示值
参数一：中点坐标
参数二：显示值
"""
plt.xticks([index for index in label_list][::20], label_list[::20]*1000)

#plt.xlabel("年份")
plt.title("frequency_distribution")
#plt.legend()     # 设置题注
# 编辑文本
#for rect in rects1:
#    height = rect.get_height()
#    plt.text(rect.get_x() + rect.get_width() / 2, height+1, str(height), ha="center", va="bottom")
plt.show()

In [ ]:
date_collect = date['t_dat'].value_counts()
dates = date['t_dat'].unique()
#print(date_collect,dates)
x = range(734)
y = list(int(i) for i in date_collect[dates[x]].values[:])
#print(x, y)
plt.plot(x,y)
plt.xticks(x[::50],[dates[i] for i in x[::50]],rotation = 'vertical')
plt.show()